In [2]:
import pandas
import numpy

In [15]:
dataDirectory = "~/.ml/data/titenic/"

In [8]:
# !mkdir -p ~/.ml/data/{dataDirectory}

In [3]:
# !kaggle competitions download -c titanic

  0%|                                               | 0.00/33.9k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 33.9k/33.9k [00:00<00:00, 3.20MB/s]


In [4]:
# !unzip titanic.zip

Archive:  titanic.zip
  inflating: train.csv               
  inflating: test.csv                
  inflating: gender_submission.csv   


In [6]:
# !mv *.csv {dataDirectory}
# !rm titanic.zip

In [18]:
trainingDataCSV = f"{dataDirectory}train.csv"
submissionDataCSV = f"{dataDirectory}test.csv"

In [19]:
dataDF = pandas.read_csv(trainingDataCSV, header=0, encoding='unicode_escape')
testDataDF = pandas.read_csv(submissionDataCSV, header=0, encoding='unicode_escape')

In [20]:
dataDF.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [29]:
object_cols = [col for col in dataDF.columns if dataDF[col].dtype == "object"]

In [30]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: dataDF[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Sex', 2), ('Embarked', 3), ('Cabin', 147), ('Ticket', 681), ('Name', 891)]

In [22]:
dataDF.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [21]:
testDataDF.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [65]:
categoricalVariables = ['Cabin',]
oneHotEncodeColumns = ['Sex', 'Embarked',]
continuousVariables = ['Pclass', 'Age', 'SibSp','Parch']

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [161]:
from sklearn.model_selection import cross_val_score

In [164]:
from sklearn.metrics import mean_absolute_error

In [157]:
imputy = SimpleImputer(strategy='constant')
allCategories = [cleanedDF[column].unique().tolist() + ['missing_value'] for column in categoricalVariables]

In [158]:
continuousTransformer = SimpleImputer(strategy='constant')
categoricalTransformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant')),
    ('encode', OrdinalEncoder(categories=allCategories))
])
oneHotTransformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant')),
    ('encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

dataPreProcessor = ColumnTransformer(transformers=[
    ('continuous', continuousTransformer, continuousVariables),
    ('oneHot', oneHotTransformer, oneHotEncodeColumns),
    ('categorical', categoricalTransformer, categoricalVariables)
])

In [162]:
xDataset = cleanedDF[oneHotEncodeColumns + categoricalVariables + continuousVariables]

In [163]:
yDataSet = cleanedDF.Survived

In [167]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0, n_estimators=1000)

In [168]:
modelPipeline = Pipeline(steps=[
    ('preProcessor', dataPreProcessor),
    ('model', model)
])

In [169]:
losses = -1 * cross_val_score(modelPipeline,
                             xDataset,
                             yDataSet,
                             cv=5,
                             scoring='neg_mean_absolute_error')

In [170]:
losses

array([0.28509513, 0.245425  , 0.24641535, 0.24713195, 0.23425371])